###  Two Box Variable Stoichiometry and Nitrogen Fixation
--- 

In this notebook I will write simulations to explore the role of deviations from Redfield on the nutrient stoichiometry of the ocean and the biota. The starting point will be Toby Tirrel's two box ocean model which predicts that nitrogen fixers cause the ocean to converge to the Redfield ratio. We will initially study the implications of allowing different N:P ratios in Nitrogen Fixers and phytoplankton, and then we will explore increasing the number of boxes.


In [1]:

from scipy.integrate import ode

%pylab inline

import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
matplotlib.use("Agg")
import scipy.io as sio
import scipy.signal
from pylab import *

asin=np.arcsin
sqrt=np.sqrt
sin=np.sin
cos=np.cos
pi=np.pi
exp=np.exp
tan=np.tan
I=1j
inf=np.inf

hilbert=scipy.signal.hilbert


Populating the interactive namespace from numpy and matplotlib


/usr/lib/pymodules/python2.7/matplotlib/__init__.py:1173: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
### Here are important constants

# Relative sizes of the high and low lattitude boxes

HLSize = .15
LLSize = .85

SD = 500.0*LLSize   # Surface layer depth in meters
DD = 3230.0  # Deep layer depth in meters

HD = 500*HLSize


K = 3.0 # Mixing coefficient in meters/year
KH = 10.0 # Mixing coefficient between high and deep boxes

SR = .95 # Fraction of TPP regenerated above 500m
SF = .002 # Fraction of TPP permanently incorporated into sediments
DR = .048 # Fraction of TPP regenerated below 500m
DN = .015 # Fraction of TPP of N that is regenerated via denitrification to N2
RP = 2.0e-4 # River input of dissolved P in mol P/m^2/year
RN = 6.0e-3 # River input of dissolved N in mol N/m^2/year
AN = 7.5e-3 # Atmospheric input of N in molN/m^2/year
RPhyto = 25.0 # N:P ratio in organic matter from phytoplankton
RPHigh = 10.0 # N:P ratio in high lattitude phytoplankton

RNFixers = 50.0 # N:P ratio in Nitrogen Fixers
muHL = 300.0 # Growth rate of high lattitude phytoplankton
muNF = 90.6 # Maximum growth rate of nitrogen fixing bacteria in units of 1/year
muPhyto = 91.25 # Maximum growth rate of phytoplankton in units of 1/year
PH = 3.0e-5 # Half saturation constant for Phosphate in mol P /m^3
NH = 5.0e-4   # Half saturation constant for Nitrate in mol N /m^3

PHHigh = 3.0e-4
NHHigh = 5.0e-3

M = 73.0    # Mortality in 1/year
MH = 110.0  # Hihgh Lattitude Mortality

### Next are the functions that define the right hand side of the system of equations

# We first define the dynamical variables:

# NFixer stands for the population of nitrogen fixers
# PhytoP stands to phytoplankton
# PS stands for the surface phosphate
# NS stands for surface nitrate
# ND stands for deep nitrate
# PD stands for deep phosphate


###

def dNFixer(NFixer,PS):
    return muNF*PS/(PS+PH)*NFixer - M*NFixer

def dPhytoP(PhytoP,PS,NS):
    return muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP - M*PhytoP

def dPS(PS,NS,PD,ND,PhytoP,NFixer):
    return -muNF*PS/(PS+PH)*NFixer/RNFixers+M*(SR)*NFixer/RNFixers+M*(SR)*PhytoP/RPhyto-muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP/RPhyto+K*(PD-PS)/SD/LLSize+RP*LLSize/SD  

def dPSH(PSH,NSH,PD,ND,PhytoPH):
    return MH*SR*PhytoPH/RPHigh-muHL*min([PSH/(PSH+PHHigh),NSH/(NSH+NHHigh)])*PhytoPH/RPHigh+KH*(PD-PSH)/HD*HLSize+RP*HLSize/HD  



def dNS(PS,NS,PD,ND,PhytoP,NFixer):
    return -muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP + M*(SR-0.75*DN)*NFixer + M*(SR-0.75*DN)*PhytoP +K*(ND-NS)/SD*LLSize+(RN+AN)*LLSize/SD

def dNSH(PSH,NSH,PD,ND,PhytoPH):
    return -muHL*min([PSH/(PSH+PHHigh),NSH/(NSH+NHHigh)])*PhytoPH+MH*(SR-0.75*DN)*PhytoPH+KH*(ND-NSH)/HD*HLSize+(RN+AN)*HLSize/HD                                   
    

def dPD(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer):
    return M*DR*NFixer/RNFixers*SD/DD+M*DR*PhytoP/RPhyto*SD/DD+MH*DR*PhytoPH/RPHigh*HD/DD - K*LLSize*(PD-PS)/DD- KH*HLSize*(PD-PSH)/DD  

def dND(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer):
    return M*(DR-0.25*DN)*NFixer*SD/DD+M*(DR-0.25*DN)*PhytoP*SD/DD-K*LLSize*(ND-NS)/DD-KH*HLSize*(ND-NSH)/DD+MH*(DR-0.25*DN)*PhytoPH*HD/DD


def dPhytoPHigh(PhytoPH,PSH,NSH):
    
    return muHL*min([PSH/(PSH+PHHigh),NSH/(NSH+NHHigh)])*PhytoPH - MH*PhytoPH



def dState(t,state):
    PS=state[0]
    PSH=state[1]
    NS=state[2]
    NSH=state[3]
    PD=state[4]
    ND=state[5]
    PhytoP=state[6]
    PhytoPH = state[7]
    NFixer=state[8]
    if PhytoP<0:
        PhytoP=0
    if NFixer <0 :
        NFixer=0
    if PhytoPH<0:
        PhytoPH=0
    
    dS = np.zeros(9)
    dS[0] = dPS(PS,NS,PD,ND,PhytoP,NFixer)
    dS[1] = dPSH(PSH,NSH,PD,ND,PhytoPH)
    dS[2] = dNS(PS,NS,PD,ND,PhytoP,NFixer)
    dS[3] = dNSH(PSH,NSH,PD,ND,PhytoPH)
    dS[4] = dPD(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer)
    dS[5] = dND(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer)
    dS[6] = dPhytoP(PhytoP,PS,NS)
    dS[7] = dPhytoPHigh(PhytoPH,PSH,NSH)
    dS[8] = dNFixer(NFixer,PS)
    return dS

###










In [3]:
### In this box we will define functions that determine the equilibrium state of the system
# This will be under the assumption that N is limiting in the high lattitude box

PSEquilibrium = M*PH/(muNF-M)
NSEquilibrium = M*NH/(muPhyto-M)
NHEquilibrium = M*NHHigh/(muHL-MH)

# Next we solve for PhytoPEquilibrium, NFixerEquilibrium, and PhytoPHighEquilibrium as functions of the 
# True PD and ND values

def PhytoPEq(PD,ND):
    





PhytoPEquilibrium = 1.0/(M*SD)*(RN+AN+RP*RNFixers*(1-SF-DN)/SF)/(SF+DN+RNFixers/RPhyto*(1-SF-DN))
NFixerEquilibrium = 1.0/(M*SD)*(RN+AN-RPhyto*RP*(SF+DN)/SF)/(DN+SF-1-RPhyto/RNFixers*(SF+DN))

PDEquilibrium = PSEquilibrium+M*DR*SD/K*(NFixerEquilibrium/RNFixers+PhytoPEquilibrium/RPhyto)
NDEquilibrium = NSEquilibrium+M*(DR-0.25*DN)*SD/K*(NFixerEquilibrium+PhytoPEquilibrium)

RS = NSEquilibrium/PSEquilibrium
RD = NDEquilibrium/PDEquilibrium



In [4]:
vecStart=np.array([2.0e-3,2.0e-3,3.2e-2,3.2e-2,2.0e-3,3.2e-2,1e-6,1e-6,1e-8])

In [41]:


rr = ode(dState).set_integrator('lsoda')
rr.set_initial_value(vecStart,0)
rr.t
t1 = 10000
dt = 2e-3
while (rr.successful() and rr.t < t1):
    rr.integrate(rr.t+dt)
    state = rr.y
    PS=state[0]
    PSH=state[1]
    NS=state[2]
    NSH=state[3]
    PD=state[4]
    ND=state[5]
    PhytoP=state[6]
    PhytoPH = state[7]
    NFixer=state[8]
    
    dS = np.zeros(9)
    if rr.y[7] <1e-15:
        print 'HERE'
        rr.y[7] = 0
        
    if PhytoP < 1e-15:
        rr.y[6]=0
    if NFixer < 1e-15:
        rr.y[8] = 1e-15
        rr.set_initial_value(rr.y)
    
    dS[0] = dPS(PS,NS,PD,ND,PhytoP,NFixer)
    dS[1] = dPSH(PSH,NSH,PD,ND,PhytoPH)
    dS[2] = dNS(PS,NS,PD,ND,PhytoP,NFixer)
    dS[3] = dNSH(PSH,NSH,PD,ND,PhytoPH)
    dS[4] = dPD(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer)
    dS[5] = dND(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer)
    dS[6] = dPhytoP(PhytoP,PS,NS)
    dS[7] = dPhytoPHigh(PhytoPH,PSH,NSH)
    dS[8] = dNFixer(NFixer,PS)
    M*(DR-0.25*DN)*NFixer*SD/DD+M*(DR-0.25*DN)*PhytoP*SD/DD-K*(ND-NS)/DD-KH*(ND-NSH)/DD+MH*(DR-0.25*DN)*PhytoPH*HD/DD
    #if NFixer<1e-14:
        #print PS, NFixer, dS[8], rr.t
    
    
    
    
    





In [22]:
dState(0,rr.y)

array([ -6.79627565e-08,   1.67764893e-11,  -3.90578240e-04,
         1.17114572e-05,   8.20144735e-07,   2.09403521e-05,
         8.10557622e-07,   2.97290137e-08,  -1.32913514e-08])

In [39]:
rr.y

array([  1.20050739e-04,   1.73684862e-04,   6.90814591e-03,
         3.49241340e-02,   3.25463344e-03,   7.21460899e-02,
         1.81051605e-04,   1.12756365e-04,   2.90958691e-08])

In [40]:
rr.y[5]/rr.y[4]



22.167193697388424

In [123]:
(RPHigh*HLSize*rr.y[7]+RPhyto*LLSize*rr.y[6]+LLSize*RNFixers*rr.y[8])/(HLSize*rr.y[7]+LLSize*rr.y[6]+LLSize*rr.y[8])


21.61335626976938

In [87]:
(RP*(1-SR-SF)/(K*SF))-M*DR*SD/K*(PhytoAlt/RPhyto+NFixerAlt/RPhyto)

NameError: name 'PhytoAlt' is not defined

In [32]:
dNFixer(1,1.25e-4)

0.06451612903225623

In [37]:
dPhytoP(1.0,1.25e-4,6.818e-3)

0.5887096774193594

In [132]:
RP/(M*SD*SF)

2.7397260273972604e-06

In [133]:
PhytoAlt+NFixerAlt-((1/(M*SD))*(RP*RPhyto*(1-SF-DN)/SF+RN+AN)+(1/(M*SD))*(RP*RPhyto*(SF+DN)-RN-AN))

0.0

In [138]:
1.0/(M*SD)*(RN+AN+RP*(1-SF-DN)/SF)/(SF+DN+RNFixers/RPhyto*(1-SF-DN))

3.063013698630137e-06

In [139]:
PhytoAlt

4.346027397260274e-05

In [141]:
(SF+DN+RNFixers/RPhyto*(1-SF-DN))

1.0

In [149]:
1.0/(M*SD)*(RN+AN+RP*RPhyto*(1-SF-DN)/SF)

4.346027397260274e-05

In [144]:
(1/(M*SD))*(RP*RPhyto*(1-SF-DN)/SF+RN+AN)

4.346027397260274e-05

In [147]:
(RN+AN+RP*(1-SF-DN)/SF)

0.1118

In [148]:
(RP*RPhyto*(1-SF-DN)/SF+RN+AN)

1.5863